<a href="https://colab.research.google.com/github/KapilM26/Insider-Threat-Detection/blob/main/one_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# %%
import csv
import os
from collections import defaultdict
from datetime import datetime, time, timedelta

import pandas as pd


import os
import csv
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

def get_user_usb_data(user_id, dataset_path):
    usb_data = []
    with open(os.path.join(dataset_path, "/content/device.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            if row[2] == user_id and row[5] == "Connect":  # Check user and only "Connect" activity
                usb_data.append(row)
    return usb_data

def get_num_usb_insertions_per_week(user, usb_data):
    weekly_usb_counts = defaultdict(int)
    all_weeks = set()
    for row in usb_data:
        file_time = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")
        week = file_time.strftime("%Y-%W")
        all_weeks.add(week)
        weekly_usb_counts[week] += 1

    # Ensure all weeks are included, even with 0 count
    min_week = min(all_weeks, default=None)
    max_week = max(all_weeks, default=None)
    if min_week and max_week:
        start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
        end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

        current_date = start_date
        complete_weeks = set()

        while current_date <= end_date:
            week_str = current_date.strftime("%Y-%W")
            complete_weeks.add(week_str)
            current_date += timedelta(days=7)

        weekly_counts = {week: weekly_usb_counts.get(week, 0) for week in complete_weeks}
    else:
        weekly_counts = {}

    output_list = [[user, week, count] for week, count in sorted(weekly_counts.items())]
    return pd.DataFrame(output_list, columns=["user", "week", "num_usb_insertions"])


def get_user_exe_data(user_id, dataset_path):
    exe_data = []
    with open(os.path.join(dataset_path, "/content/file.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            if row[2] == user_id and row[4].endswith(".exe"):  # Check user and .exe files
                exe_data.append(row)
    return exe_data

def get_num_exe_per_week(user, exe_data):
    weekly_exe_counts = defaultdict(int)
    all_weeks = set()

    for row in exe_data:
        file_time = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")
        week = file_time.strftime("%Y-%W")
        all_weeks.add(week)
        weekly_exe_counts[week] += 1

    # Ensure all weeks are included, even with 0 count
    min_week = min(all_weeks, default=None)
    max_week = max(all_weeks, default=None)

    if min_week and max_week:
        start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
        end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

        current_date = start_date
        complete_weeks = set()

        while current_date <= end_date:
            week_str = current_date.strftime("%Y-%W")
            complete_weeks.add(week_str)
            current_date += timedelta(days=7)

        weekly_counts = {week: weekly_exe_counts.get(week, 0) for week in complete_weeks}
    else:
        weekly_counts = {}

    output_list = [[user, week, count] for week, count in sorted(weekly_counts.items())]
    return pd.DataFrame(output_list, columns=["user", "week", "num_exe_files"])

# %%
def get_user_logon_data(user_id, dataset_path):
    logon_data = []
    with open(os.path.join(dataset_path, "/content/logon.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row[2] == user_id:
                logon_data.append(row)
    return logon_data


# %%
def get_user_pc(logon_data):
    pc_dict = {}
    for row in logon_data:
        pc_dict[row[3]] = 1 + pc_dict.get(row[3], 0)
    user_pc = max(pc_dict, key=pc_dict.get)
    return user_pc


# %%
def get_num_other_PC_per_week(user, user_pc, logon_data):
    weekly_pc_counts = defaultdict(set)  # Dictionary to store unique PCs per week
    all_weeks = set()  # Set to track all weeks where logons occurred

    for row in logon_data:
        logon_time = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")  # Adjusted format
        week = logon_time.strftime("%Y-%W")  # Year-Week format
        all_weeks.add(week)  # Track all weeks

        if row[3] != user_pc:  # Check if PC is different from user's primary PC
            weekly_pc_counts[week].add(
                row[3]
            )  # Add PC to the week's set (unique values only)

    # Ensure all weeks are included, even with 0 count
    min_week = min(all_weeks)
    max_week = max(all_weeks)

    # Generate all weeks between min and max
    start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
    end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

    current_date = start_date
    complete_weeks = set()

    while current_date <= end_date:
        week_str = current_date.strftime("%Y-%W")
        complete_weeks.add(week_str)
        current_date += timedelta(days=7)

    # Ensure every week has a count (0 if no other PCs were accessed)
    weekly_counts = {
        week: len(weekly_pc_counts[week]) if week in weekly_pc_counts else 0
        for week in complete_weeks
    }

    # Convert to DataFrame
    output_list = [[user, week, count] for week, count in sorted(weekly_counts.items())]
    return pd.DataFrame(output_list, columns=["user", "week", "num_other_pc"])


def get_after_hours_logons(
    logon_data, user, business_start=time(9, 0, 0), business_end=time(17, 0, 0)
):
    """
    Aggregates after-hours logons per week for a specified user.

    :param logon_data: List of logon events in the format [id, date, user, pc, activity]
    :param user: The specific user to filter logon events for.
    :param business_start: Datetime.time representing start of business hours.
    :param business_end: Datetime.time representing end of business hours.
    :return: DataFrame with ['user', 'week', 'after_hours_logons']
    """

    after_hours_counts = defaultdict(int)

    # Track all weeks for the user
    all_weeks = set()

    for row in logon_data:
        logon_id, timestamp, logon_user, pc, activity = row  # Unpack columns

        if (
            activity.lower() == "logon" and logon_user == user
        ):  # Only process logons for the specified user
            try:
                logon_time = datetime.strptime(timestamp, "%m/%d/%Y %H:%M:%S")
                logon_week = logon_time.strftime("%Y-%W")  # Ensure same format

                # Store this week to ensure it's included in results
                all_weeks.add(logon_week)

                # Extract only the time component
                logon_hour = logon_time.time()

                # Check if the logon occurred outside business hours
                if logon_hour < business_start or logon_hour >= business_end:
                    after_hours_counts[logon_week] += 1

            except ValueError:
                continue  # Skip invalid timestamps

    # Ensure all weeks in range are included (like `get_num_other_PC_per_week`)
    if all_weeks:
        min_week = min(all_weeks)
        max_week = max(all_weeks)

        # Generate all weeks in range
        start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
        end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

        current_date = start_date
        complete_weeks = set()

        while current_date <= end_date:
            week_str = current_date.strftime("%Y-%W")
            complete_weeks.add(week_str)
            current_date += timedelta(days=7)

        # Fill in missing weeks with 0
        after_hours_counts = {
            week: after_hours_counts.get(week, 0) for week in complete_weeks
        }

    # Convert to DataFrame
    result_data = [
        (user, week, after_hours_counts[week])
        for week in sorted(after_hours_counts.keys())
    ]
    after_hours_df = pd.DataFrame(
        result_data, columns=["user", "week", "after_hours_logons"]
    )

    return after_hours_df


# %%
def find_insider_answers_file(user, insider_root):
    """
    Recursively searches for the insider CSV file for the given user in the `insider_root` directory.

    :param user: The user ID (e.g., "CWW1120")
    :param insider_root: The root folder containing multiple r5.2-* subfolders.
    :return: The full path to the user's insider CSV file if found, else None.
    """
    for root, _, files in os.walk(insider_root):
        for file in files:
            if file.startswith(f"/content/r5.2-") and file.endswith(
                f"-{user}.csv"
            ):  # Match user file format
                return os.path.join(root, file)  # Return full file path if found
    return None  # Return None if no file is found


def extract_weeks_from_csv(file_path):
    """
    Reads a CSV file using `csv.reader` and extracts unique weeks from the timestamps (3rd column).

    :param file_path: Path to the insider CSV file.
    :return: A set of detected `Year-Week` values.
    """
    insider_weeks = set()

    try:
        with open(file_path, mode="r", newline="", encoding="utf-8") as file:
            reader = csv.reader(file)
            for row in reader:
                if len(row) < 3:  # Ensure the timestamp column exists
                    continue
                try:
                    logon_time = datetime.strptime(
                        row[2], "%m/%d/%Y %H:%M:%S"
                    )  # Parse timestamp
                    week = logon_time.strftime("%Y-%W")  # Convert to Year-Week format
                    insider_weeks.add(week)
                except ValueError:
                    continue  # Skip rows with invalid timestamps
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

    return insider_weeks


def label_insider_weeks(df, user, insider_root):
    """
    Adds an 'insider' column to the DataFrame by checking if the user's week exists in their insider file.

    :param df: DataFrame containing ['user', 'week', 'num_other_pc']
    :param user: The user ID for whom the dataframe is filtered.
    :param insider_root: Path to the folder containing multiple r5.2-* subfolders.
    :return: DataFrame with an 'insider' column.
    """

    # Locate the user's insider file
    insider_file = find_insider_answers_file(user, insider_root)

    # If no insider file exists for the user, mark all weeks as 0 (not insider)
    if not insider_file:
        df["insider"] = 0
        return df

    # Extract weeks from the insider CSV file
    insider_weeks = extract_weeks_from_csv(insider_file)

    # Label insider weeks in the user's dataframe
    df["insider"] = df["week"].apply(lambda w: 1 if w in insider_weeks else 0)

    return df


def combine_user_feature_data(user, dataset_path, insider_root):
    # Get data from different feature functions
    logon_data = get_user_logon_data(user, dataset_path)
    user_pc = get_user_pc(logon_data)
    num_other_pc = get_num_other_PC_per_week(user, user_pc, logon_data)
    after_hours_logons = get_after_hours_logons(logon_data, user)

    exe_data = get_user_exe_data(user, dataset_path)
    num_exe_files = get_num_exe_per_week(user, exe_data)

    usb_data = get_user_usb_data(user, dataset_path)
    num_usb = get_num_usb_insertions_per_week(user, usb_data)

    # Extract relevant columns
    after_hours_df = after_hours_logons[["week", "after_hours_logons"]]
    exe_df         = num_exe_files[["week", "num_exe_files"]]
    usb_df         = num_usb[["week", "num_usb_insertions"]]
    other_pc_df    = num_other_pc[["week", "num_other_pc"]]

    # Merge all dataframes on "week" using an outer join
    merged_df = after_hours_df.merge(exe_df, on="week", how="outer") \
                              .merge(usb_df, on="week", how="outer") \
                              .merge(other_pc_df, on="week", how="outer")

    # Replace NaN with 0 in all feature columns
    merged_df.fillna(0, inplace=True)

    # Add user column
    merged_df.insert(0, "user", user)
    labeled_df = label_insider_weeks(merged_df, user, insider_root)
    return labeled_df

# Example usage
dataset_path = os.path.join("Insider threat dataset", "r5.2")
user = "MIB0203"
file_path = "/content/MIB0203.csv"  # Change to your actual file path
final_df = pd.read_csv(file_path)
# final_df = combine_user_feature_data(user, dataset_path, insider_root)
print(final_df)

FileNotFoundError: [Errno 2] No such file or directory: '/content/MIB0203.csv'

In [23]:

# # Access the 'num_other_pc' data from the 'final_df' DataFrame
# num_other_pc_data = final_df[['num_other_pc']]

# # Initialize the Isolation Forest model
# iso_forest = IsolationForest(contamination='auto', random_state=42)

# # Fit the model
# iso_forest.fit(num_other_pc_data)

# # Predict anomalies and add results to the 'final_df' DataFrame
# final_df['anomaly'] = iso_forest.predict(num_other_pc_data)
# final_df['anomaly_score'] = iso_forest.decision_function(num_other_pc_data)

# # Convert anomaly labels to boolean (1 for anomaly, -1 for normal)
# final_df['anomaly'] = final_df['anomaly'].apply(lambda x: 1 if x == -1 else 0)

# # Display the rows where anomalies were detected
# print(final_df[final_df['anomaly'] == 1])

In [61]:
print(pd.Series(y_pred).value_counts())


0    29
1    15
Name: count, dtype: int64


In [81]:
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

# Select Features
feature_cols = ["after_hours_logons", "num_exe_files", "num_usb_insertions", "num_other_pc"]
X = final_df[feature_cols]
y_true = final_df["insider"]

# Normalize Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train One-Class SVM with a Higher `nu`
oc_svm = OneClassSVM(kernel="rbf", gamma="scale", nu=0.4)  # Increased from 0.2 to 0.4
oc_svm.fit(X_scaled)

# Predict Anomalies
decision_scores = oc_svm.decision_function(X_scaled)

# Lower the Threshold to Detect More Anomalies
threshold = np.percentile(decision_scores, 80)  # Changed from 90 to 80
y_pred = [1 if score < threshold else 0 for score in decision_scores]

# Calculate Metrics
precision = precision_score(y_true, y_pred, pos_label=1, zero_division=1)
recall = recall_score(y_true, y_pred, pos_label=1, zero_division=1)
f1 = f1_score(y_true, y_pred, pos_label=1, zero_division=1)

# Print Results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Debug: Print Prediction Counts
print("Predicted Labels:\n", pd.Series(y_pred).value_counts())
print("True Labels:\n", pd.Series(y_true).value_counts())


Precision: 0.0588
Recall: 1.0000
F1-score: 0.1111
Predicted Labels:
 1    34
0    10
Name: count, dtype: int64
True Labels:
 insider
0    42
1     2
Name: count, dtype: int64


In [8]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

#  Select Features for Training
feature_cols = ["after_hours_logons", "num_exe_files", "num_usb_insertions", "num_other_pc"]
X = final_df[feature_cols]
y_true = final_df["insider"]  # Ground truth (0 = normal, 1 = anomaly)

# Normalize the Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#  Train One-Class SVM
oc_svm = OneClassSVM(kernel="rbf", gamma="scale", nu=0.9)  # Adjust nu for anomaly proportion
oc_svm.fit(X_scaled)

# Predict Anomalies
decision_scores = oc_svm.decision_function(X_scaled)  # Get decision function scores
threshold = np.percentile(decision_scores, 90)  # Detect top 10% as anomalies
y_pred = [1 if score < threshold else 0 for score in decision_scores]

#  Calculate Evaluation Metrics
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
f1 = f1_score(y_true, y_pred, pos_label=1)

#  Print Results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


NameError: name 'final_df' is not defined

In [7]:
# Define different threshold values for anomaly detection
thresholds = np.percentile(decision_scores, [5, 10, 15, 20, 25])  # Try different percentiles

# Loop through each threshold and compute precision, recall, F1-score
for threshold in thresholds:
  y_pred = np.where(decision_scores < threshold, 1, 0)  # Label as "1" if below threshold (anomaly)

  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)

  print(f"Threshold = {threshold:.4f}: Precision = {precision:.4f}, Recall = {recall:.4f}, F1 Score = {f1:.4f}")

NameError: name 'np' is not defined

In [96]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor  # Import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

# Define your possible contamination values (thresholds)
contamination_values = [0.01, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15]

# Loop through each contamination value and calculate performance metrics
for contamination in contamination_values:
    lof = LocalOutlierFactor(n_neighbors=20, contamination=contamination)

    # Fit LOF model and get predictions
    y_pred = lof.fit_predict(X)  # Assuming X_train is your feature matrix
    y_pred = [1 if pred == -1 else 0 for pred in y_pred]  # Convert LOF output (-1 for outliers, 1 for inliers)

    # Calculate precision, recall, and F1 score for insider=1 (malicious)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Contamination = {contamination}: Precision = {precision:.4f}, Recall = {recall:.4f}, F1 Score = {f1:.4f}")

Contamination = 0.01: Precision = 1.0000, Recall = 0.5000, F1 Score = 0.6667
Contamination = 0.025: Precision = 1.0000, Recall = 0.5000, F1 Score = 0.6667
Contamination = 0.05: Precision = 1.0000, Recall = 1.0000, F1 Score = 1.0000
Contamination = 0.075: Precision = 0.6667, Recall = 1.0000, F1 Score = 0.8000
Contamination = 0.1: Precision = 0.5000, Recall = 1.0000, F1 Score = 0.6667
Contamination = 0.125: Precision = 0.4000, Recall = 1.0000, F1 Score = 0.5714
Contamination = 0.15: Precision = 0.3333, Recall = 1.0000, F1 Score = 0.5000


In [2]:
# Define different threshold values for anomaly detection
    thresholds = np.percentile(decision_scores, [5, 10, 15, 20, 25])  # Try different percentiles

    # Loop through each threshold and compute precision, recall, F1-score
    for threshold in thresholds:
        y_pred = np.where(decision_scores < threshold, 1, 0)  # Label as "1" if below threshold (anomaly)

        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        print(f"Threshold = {threshold:.4f}: Precision = {precision:.4f}, Recall = {recall:.4f}, F1 Score = {f1:.4f}")

IndentationError: unexpected indent (<ipython-input-2-c0566b08f270>, line 2)

In [79]:
final_df[final_df["num_usb_insertions"]!=0].head()

,user,week,after_hours_logons,num_exe_files,num_usb_insertions,num_other_pc,insider
40,MIB0203,2010-41,7,0,1.0,0,1
41,MIB0203,2010-42,7,0,1.0,1,1


In [85]:
# %%
import csv
import os
from collections import defaultdict
from datetime import datetime, time, timedelta

import pandas as pd


import os
import csv
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

def get_user_usb_data(user_id, dataset_path):
    usb_data = []
    with open(os.path.join(dataset_path, "/content/device.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            if row[2] == user_id and row[5] == "Connect":  # Check user and only "Connect" activity
                usb_data.append(row)
    return usb_data

def get_num_usb_insertions_per_week(user, usb_data):
    weekly_usb_counts = defaultdict(int)
    all_weeks = set()
    for row in usb_data:
        file_time = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")
        week = file_time.strftime("%Y-%W")
        all_weeks.add(week)
        weekly_usb_counts[week] += 1

    # Ensure all weeks are included, even with 0 count
    min_week = min(all_weeks, default=None)
    max_week = max(all_weeks, default=None)
    if min_week and max_week:
        start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
        end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

        current_date = start_date
        complete_weeks = set()

        while current_date <= end_date:
            week_str = current_date.strftime("%Y-%W")
            complete_weeks.add(week_str)
            current_date += timedelta(days=7)

        weekly_counts = {week: weekly_usb_counts.get(week, 0) for week in complete_weeks}
    else:
        weekly_counts = {}

    output_list = [[user, week, count] for week, count in sorted(weekly_counts.items())]
    return pd.DataFrame(output_list, columns=["user", "week", "num_usb_insertions"])


def get_user_exe_data(user_id, dataset_path):
    exe_data = []
    with open(os.path.join(dataset_path, "/content/file.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            if row[2] == user_id and row[4].endswith(".exe"):  # Check user and .exe files
                exe_data.append(row)
    return exe_data

def get_num_exe_per_week(user, exe_data):
    weekly_exe_counts = defaultdict(int)
    all_weeks = set()

    for row in exe_data:
        file_time = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")
        week = file_time.strftime("%Y-%W")
        all_weeks.add(week)
        weekly_exe_counts[week] += 1

    # Ensure all weeks are included, even with 0 count
    min_week = min(all_weeks, default=None)
    max_week = max(all_weeks, default=None)

    if min_week and max_week:
        start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
        end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

        current_date = start_date
        complete_weeks = set()

        while current_date <= end_date:
            week_str = current_date.strftime("%Y-%W")
            complete_weeks.add(week_str)
            current_date += timedelta(days=7)

        weekly_counts = {week: weekly_exe_counts.get(week, 0) for week in complete_weeks}
    else:
        weekly_counts = {}

    output_list = [[user, week, count] for week, count in sorted(weekly_counts.items())]
    return pd.DataFrame(output_list, columns=["user", "week", "num_exe_files"])

# %%
def get_user_logon_data(user_id, dataset_path):
    logon_data = []
    with open(os.path.join(dataset_path, "/content/logon.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row[2] == user_id:
                logon_data.append(row)
    return logon_data


# %%
def get_user_pc(logon_data):
    pc_dict = {}
    for row in logon_data:
        pc_dict[row[3]] = 1 + pc_dict.get(row[3], 0)
    user_pc = max(pc_dict, key=pc_dict.get)
    return user_pc


# %%
def get_num_other_PC_per_week(user, user_pc, logon_data):
    weekly_pc_counts = defaultdict(set)  # Dictionary to store unique PCs per week
    all_weeks = set()  # Set to track all weeks where logons occurred

    for row in logon_data:
        logon_time = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")  # Adjusted format
        week = logon_time.strftime("%Y-%W")  # Year-Week format
        all_weeks.add(week)  # Track all weeks

        if row[3] != user_pc:  # Check if PC is different from user's primary PC
            weekly_pc_counts[week].add(
                row[3]
            )  # Add PC to the week's set (unique values only)

    # Ensure all weeks are included, even with 0 count
    min_week = min(all_weeks)
    max_week = max(all_weeks)

    # Generate all weeks between min and max
    start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
    end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

    current_date = start_date
    complete_weeks = set()

    while current_date <= end_date:
        week_str = current_date.strftime("%Y-%W")
        complete_weeks.add(week_str)
        current_date += timedelta(days=7)

    # Ensure every week has a count (0 if no other PCs were accessed)
    weekly_counts = {
        week: len(weekly_pc_counts[week]) if week in weekly_pc_counts else 0
        for week in complete_weeks
    }

    # Convert to DataFrame
    output_list = [[user, week, count] for week, count in sorted(weekly_counts.items())]
    return pd.DataFrame(output_list, columns=["user", "week", "num_other_pc"])


def get_after_hours_logons(
    logon_data, user, business_start=time(9, 0, 0), business_end=time(17, 0, 0)
):
    """
    Aggregates after-hours logons per week for a specified user.

    :param logon_data: List of logon events in the format [id, date, user, pc, activity]
    :param user: The specific user to filter logon events for.
    :param business_start: Datetime.time representing start of business hours.
    :param business_end: Datetime.time representing end of business hours.
    :return: DataFrame with ['user', 'week', 'after_hours_logons']
    """

    after_hours_counts = defaultdict(int)

    # Track all weeks for the user
    all_weeks = set()

    for row in logon_data:
        logon_id, timestamp, logon_user, pc, activity = row  # Unpack columns

        if (
            activity.lower() == "logon" and logon_user == user
        ):  # Only process logons for the specified user
            try:
                logon_time = datetime.strptime(timestamp, "%m/%d/%Y %H:%M:%S")
                logon_week = logon_time.strftime("%Y-%W")  # Ensure same format

                # Store this week to ensure it's included in results
                all_weeks.add(logon_week)

                # Extract only the time component
                logon_hour = logon_time.time()

                # Check if the logon occurred outside business hours
                if logon_hour < business_start or logon_hour >= business_end:
                    after_hours_counts[logon_week] += 1

            except ValueError:
                continue  # Skip invalid timestamps

    # Ensure all weeks in range are included (like `get_num_other_PC_per_week`)
    if all_weeks:
        min_week = min(all_weeks)
        max_week = max(all_weeks)

        # Generate all weeks in range
        start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
        end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

        current_date = start_date
        complete_weeks = set()

        while current_date <= end_date:
            week_str = current_date.strftime("%Y-%W")
            complete_weeks.add(week_str)
            current_date += timedelta(days=7)

        # Fill in missing weeks with 0
        after_hours_counts = {
            week: after_hours_counts.get(week, 0) for week in complete_weeks
        }

    # Convert to DataFrame
    result_data = [
        (user, week, after_hours_counts[week])
        for week in sorted(after_hours_counts.keys())
    ]
    after_hours_df = pd.DataFrame(
        result_data, columns=["user", "week", "after_hours_logons"]
    )

    return after_hours_df


# %%
def find_insider_answers_file(user, insider_root):
    """
    Recursively searches for the insider CSV file for the given user in the `insider_root` directory.

    :param user: The user ID (e.g., "CWW1120")
    :param insider_root: The root folder containing multiple r5.2-* subfolders.
    :return: The full path to the user's insider CSV file if found, else None.
    """
    for root, _, files in os.walk(insider_root):
        for file in files:
            if file.startswith(f"/content/r5.2-") and file.endswith(
                f"-{user}.csv"
            ):  # Match user file format
                return os.path.join(root, file)  # Return full file path if found
    return None  # Return None if no file is found


def extract_weeks_from_csv(file_path):
    """
    Reads a CSV file using `csv.reader` and extracts unique weeks from the timestamps (3rd column).

    :param file_path: Path to the insider CSV file.
    :return: A set of detected `Year-Week` values.
    """
    insider_weeks = set()

    try:
        with open(file_path, mode="r", newline="", encoding="utf-8") as file:
            reader = csv.reader(file)
            for row in reader:
                if len(row) < 3:  # Ensure the timestamp column exists
                    continue
                try:
                    logon_time = datetime.strptime(
                        row[2], "%m/%d/%Y %H:%M:%S"
                    )  # Parse timestamp
                    week = logon_time.strftime("%Y-%W")  # Convert to Year-Week format
                    insider_weeks.add(week)
                except ValueError:
                    continue  # Skip rows with invalid timestamps
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

    return insider_weeks


def label_insider_weeks(df, user, insider_root):
    """
    Adds an 'insider' column to the DataFrame by checking if the user's week exists in their insider file.

    :param df: DataFrame containing ['user', 'week', 'num_other_pc']
    :param user: The user ID for whom the dataframe is filtered.
    :param insider_root: Path to the folder containing multiple r5.2-* subfolders.
    :return: DataFrame with an 'insider' column.
    """

    # Locate the user's insider file
    insider_file = find_insider_answers_file(user, insider_root)

    # If no insider file exists for the user, mark all weeks as 0 (not insider)
    if not insider_file:
        df["insider"] = 0
        return df

    # Extract weeks from the insider CSV file
    insider_weeks = extract_weeks_from_csv(insider_file)

    # Label insider weeks in the user's dataframe
    df["insider"] = df["week"].apply(lambda w: 1 if w in insider_weeks else 0)

    return df


def combine_user_feature_data(user, dataset_path, insider_root):
    # Get data from different feature functions
    logon_data = get_user_logon_data(user, dataset_path)
    user_pc = get_user_pc(logon_data)
    num_other_pc = get_num_other_PC_per_week(user, user_pc, logon_data)
    after_hours_logons = get_after_hours_logons(logon_data, user)

    exe_data = get_user_exe_data(user, dataset_path)
    num_exe_files = get_num_exe_per_week(user, exe_data)

    usb_data = get_user_usb_data(user, dataset_path)
    num_usb = get_num_usb_insertions_per_week(user, usb_data)

    # Extract relevant columns
    after_hours_df = after_hours_logons[["week", "after_hours_logons"]]
    exe_df         = num_exe_files[["week", "num_exe_files"]]
    usb_df         = num_usb[["week", "num_usb_insertions"]]
    other_pc_df    = num_other_pc[["week", "num_other_pc"]]

    # Merge all dataframes on "week" using an outer join
    merged_df = after_hours_df.merge(exe_df, on="week", how="outer") \
                              .merge(usb_df, on="week", how="outer") \
                              .merge(other_pc_df, on="week", how="outer")

    # Replace NaN with 0 in all feature columns
    merged_df.fillna(0, inplace=True)

    # Add user column
    merged_df.insert(0, "user", user)
    labeled_df = label_insider_weeks(merged_df, user, insider_root)
    return labeled_df

# Example usage
dataset_path = os.path.join("Insider threat dataset", "r5.2")
user = "ALT1465"
file_path = "/content/ALT1465.csv"  # Change to your actual file path
final_df = pd.read_csv(file_path)
# final_df = combine_user_feature_data(user, data
print(final_df)

       user     week  after_hours_logons  num_exe_files  num_usb_insertions  \
0   ALT1465  2010-01                   0              0                 0.0   
1   ALT1465  2010-02                   4              0                 0.0   
2   ALT1465  2010-03                   4              0                 0.0   
3   ALT1465  2010-04                   4              0                 0.0   
4   ALT1465  2010-05                   1              0                 0.0   
5   ALT1465  2010-06                   0              0                 0.0   
6   ALT1465  2010-07                   5              0                 0.0   
7   ALT1465  2010-08                   2              0                 0.0   
8   ALT1465  2010-09                   2              0                 0.0   
9   ALT1465  2010-10                   5              0                 0.0   
10  ALT1465  2010-11                   2              0                 0.0   
11  ALT1465  2010-12                   3            

In [89]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

#  Select Features for Training
feature_cols = ["after_hours_logons", "num_exe_files", "num_usb_insertions", "num_other_pc"]
X = final_df[feature_cols]
y_true = final_df["insider"]  # Ground truth (0 = normal, 1 = anomaly)

# Normalize the Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#  Train One-Class SVM
oc_svm = OneClassSVM(kernel="rbf", gamma="scale", nu=0.9)  # Adjust nu for anomaly proportion
oc_svm.fit(X_scaled)

# Predict Anomalies
decision_scores = oc_svm.decision_function(X_scaled)  # Get decision function scores
threshold = np.percentile(decision_scores, 90)  # Detect top 10% as anomalies
y_pred = [1 if score < threshold else 0 for score in decision_scores]

#  Calculate Evaluation Metrics
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
f1 = f1_score(y_true, y_pred, pos_label=1)

#  Print Results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Precision: 0.0690
Recall: 1.0000
F1-score: 0.1290


In [92]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor  # Import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

# Define your possible contamination values (thresholds)
contamination_values = [0.01, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15]

# Loop through each contamination value and calculate performance metrics
for contamination in contamination_values:
    lof = LocalOutlierFactor(n_neighbors=20, contamination=contamination)

    # Fit LOF model and get predictions
    y_pred = lof.fit_predict(X)  # Assuming X_train is your feature matrix
    y_pred = [1 if pred == -1 else 0 for pred in y_pred]  # Convert LOF output (-1 for outliers, 1 for inliers)

    # Calculate precision, recall, and F1 score for insider=1 (malicious)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Contamination = {contamination}: Precision = {precision:.4f}, Recall = {recall:.4f}, F1 Score = {f1:.4f}")

Contamination = 0.01: Precision = 1.0000, Recall = 0.5000, F1 Score = 0.6667
Contamination = 0.025: Precision = 1.0000, Recall = 0.5000, F1 Score = 0.6667
Contamination = 0.05: Precision = 1.0000, Recall = 1.0000, F1 Score = 1.0000
Contamination = 0.075: Precision = 0.6667, Recall = 1.0000, F1 Score = 0.8000
Contamination = 0.1: Precision = 0.5000, Recall = 1.0000, F1 Score = 0.6667
Contamination = 0.125: Precision = 0.4000, Recall = 1.0000, F1 Score = 0.5714
Contamination = 0.15: Precision = 0.3333, Recall = 1.0000, F1 Score = 0.5000


In [93]:
final_df[final_df["num_usb_insertions"]!=0].head()

,user,week,after_hours_logons,num_exe_files,num_usb_insertions,num_other_pc,insider
31,ALT1465,2010-32,3,0,3.0,1,1
32,ALT1465,2010-33,6,0,3.0,2,1
33,ALT1465,2010-34,2,0,3.0,1,0


In [97]:
# %%
import csv
import os
from collections import defaultdict
from datetime import datetime, time, timedelta

import pandas as pd


import os
import csv
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

def get_user_usb_data(user_id, dataset_path):
    usb_data = []
    with open(os.path.join(dataset_path, "/content/device.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            if row[2] == user_id and row[5] == "Connect":  # Check user and only "Connect" activity
                usb_data.append(row)
    return usb_data

def get_num_usb_insertions_per_week(user, usb_data):
    weekly_usb_counts = defaultdict(int)
    all_weeks = set()
    for row in usb_data:
        file_time = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")
        week = file_time.strftime("%Y-%W")
        all_weeks.add(week)
        weekly_usb_counts[week] += 1

    # Ensure all weeks are included, even with 0 count
    min_week = min(all_weeks, default=None)
    max_week = max(all_weeks, default=None)
    if min_week and max_week:
        start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
        end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

        current_date = start_date
        complete_weeks = set()

        while current_date <= end_date:
            week_str = current_date.strftime("%Y-%W")
            complete_weeks.add(week_str)
            current_date += timedelta(days=7)

        weekly_counts = {week: weekly_usb_counts.get(week, 0) for week in complete_weeks}
    else:
        weekly_counts = {}

    output_list = [[user, week, count] for week, count in sorted(weekly_counts.items())]
    return pd.DataFrame(output_list, columns=["user", "week", "num_usb_insertions"])


def get_user_exe_data(user_id, dataset_path):
    exe_data = []
    with open(os.path.join(dataset_path, "/content/file.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header
        for row in reader:
            if row[2] == user_id and row[4].endswith(".exe"):  # Check user and .exe files
                exe_data.append(row)
    return exe_data

def get_num_exe_per_week(user, exe_data):
    weekly_exe_counts = defaultdict(int)
    all_weeks = set()

    for row in exe_data:
        file_time = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")
        week = file_time.strftime("%Y-%W")
        all_weeks.add(week)
        weekly_exe_counts[week] += 1

    # Ensure all weeks are included, even with 0 count
    min_week = min(all_weeks, default=None)
    max_week = max(all_weeks, default=None)

    if min_week and max_week:
        start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
        end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

        current_date = start_date
        complete_weeks = set()

        while current_date <= end_date:
            week_str = current_date.strftime("%Y-%W")
            complete_weeks.add(week_str)
            current_date += timedelta(days=7)

        weekly_counts = {week: weekly_exe_counts.get(week, 0) for week in complete_weeks}
    else:
        weekly_counts = {}

    output_list = [[user, week, count] for week, count in sorted(weekly_counts.items())]
    return pd.DataFrame(output_list, columns=["user", "week", "num_exe_files"])

# %%
def get_user_logon_data(user_id, dataset_path):
    logon_data = []
    with open(os.path.join(dataset_path, "/content/logon.csv"), "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row[2] == user_id:
                logon_data.append(row)
    return logon_data


# %%
def get_user_pc(logon_data):
    pc_dict = {}
    for row in logon_data:
        pc_dict[row[3]] = 1 + pc_dict.get(row[3], 0)
    user_pc = max(pc_dict, key=pc_dict.get)
    return user_pc


# %%
def get_num_other_PC_per_week(user, user_pc, logon_data):
    weekly_pc_counts = defaultdict(set)  # Dictionary to store unique PCs per week
    all_weeks = set()  # Set to track all weeks where logons occurred

    for row in logon_data:
        logon_time = datetime.strptime(row[1], "%m/%d/%Y %H:%M:%S")  # Adjusted format
        week = logon_time.strftime("%Y-%W")  # Year-Week format
        all_weeks.add(week)  # Track all weeks

        if row[3] != user_pc:  # Check if PC is different from user's primary PC
            weekly_pc_counts[week].add(
                row[3]
            )  # Add PC to the week's set (unique values only)

    # Ensure all weeks are included, even with 0 count
    min_week = min(all_weeks)
    max_week = max(all_weeks)

    # Generate all weeks between min and max
    start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
    end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

    current_date = start_date
    complete_weeks = set()

    while current_date <= end_date:
        week_str = current_date.strftime("%Y-%W")
        complete_weeks.add(week_str)
        current_date += timedelta(days=7)

    # Ensure every week has a count (0 if no other PCs were accessed)
    weekly_counts = {
        week: len(weekly_pc_counts[week]) if week in weekly_pc_counts else 0
        for week in complete_weeks
    }

    # Convert to DataFrame
    output_list = [[user, week, count] for week, count in sorted(weekly_counts.items())]
    return pd.DataFrame(output_list, columns=["user", "week", "num_other_pc"])


def get_after_hours_logons(
    logon_data, user, business_start=time(9, 0, 0), business_end=time(17, 0, 0)
):
    """
    Aggregates after-hours logons per week for a specified user.

    :param logon_data: List of logon events in the format [id, date, user, pc, activity]
    :param user: The specific user to filter logon events for.
    :param business_start: Datetime.time representing start of business hours.
    :param business_end: Datetime.time representing end of business hours.
    :return: DataFrame with ['user', 'week', 'after_hours_logons']
    """

    after_hours_counts = defaultdict(int)

    # Track all weeks for the user
    all_weeks = set()

    for row in logon_data:
        logon_id, timestamp, logon_user, pc, activity = row  # Unpack columns

        if (
            activity.lower() == "logon" and logon_user == user
        ):  # Only process logons for the specified user
            try:
                logon_time = datetime.strptime(timestamp, "%m/%d/%Y %H:%M:%S")
                logon_week = logon_time.strftime("%Y-%W")  # Ensure same format

                # Store this week to ensure it's included in results
                all_weeks.add(logon_week)

                # Extract only the time component
                logon_hour = logon_time.time()

                # Check if the logon occurred outside business hours
                if logon_hour < business_start or logon_hour >= business_end:
                    after_hours_counts[logon_week] += 1

            except ValueError:
                continue  # Skip invalid timestamps

    # Ensure all weeks in range are included (like `get_num_other_PC_per_week`)
    if all_weeks:
        min_week = min(all_weeks)
        max_week = max(all_weeks)

        # Generate all weeks in range
        start_date = datetime.strptime(min_week + "-1", "%Y-%W-%w")
        end_date = datetime.strptime(max_week + "-1", "%Y-%W-%w")

        current_date = start_date
        complete_weeks = set()

        while current_date <= end_date:
            week_str = current_date.strftime("%Y-%W")
            complete_weeks.add(week_str)
            current_date += timedelta(days=7)

        # Fill in missing weeks with 0
        after_hours_counts = {
            week: after_hours_counts.get(week, 0) for week in complete_weeks
        }

    # Convert to DataFrame
    result_data = [
        (user, week, after_hours_counts[week])
        for week in sorted(after_hours_counts.keys())
    ]
    after_hours_df = pd.DataFrame(
        result_data, columns=["user", "week", "after_hours_logons"]
    )

    return after_hours_df


# %%
def find_insider_answers_file(user, insider_root):
    """
    Recursively searches for the insider CSV file for the given user in the `insider_root` directory.

    :param user: The user ID (e.g., "CWW1120")
    :param insider_root: The root folder containing multiple r5.2-* subfolders.
    :return: The full path to the user's insider CSV file if found, else None.
    """
    for root, _, files in os.walk(insider_root):
        for file in files:
            if file.startswith(f"/content/r5.2-") and file.endswith(
                f"-{user}.csv"
            ):  # Match user file format
                return os.path.join(root, file)  # Return full file path if found
    return None  # Return None if no file is found


def extract_weeks_from_csv(file_path):
    """
    Reads a CSV file using `csv.reader` and extracts unique weeks from the timestamps (3rd column).

    :param file_path: Path to the insider CSV file.
    :return: A set of detected `Year-Week` values.
    """
    insider_weeks = set()

    try:
        with open(file_path, mode="r", newline="", encoding="utf-8") as file:
            reader = csv.reader(file)
            for row in reader:
                if len(row) < 3:  # Ensure the timestamp column exists
                    continue
                try:
                    logon_time = datetime.strptime(
                        row[2], "%m/%d/%Y %H:%M:%S"
                    )  # Parse timestamp
                    week = logon_time.strftime("%Y-%W")  # Convert to Year-Week format
                    insider_weeks.add(week)
                except ValueError:
                    continue  # Skip rows with invalid timestamps
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

    return insider_weeks


def label_insider_weeks(df, user, insider_root):
    """
    Adds an 'insider' column to the DataFrame by checking if the user's week exists in their insider file.

    :param df: DataFrame containing ['user', 'week', 'num_other_pc']
    :param user: The user ID for whom the dataframe is filtered.
    :param insider_root: Path to the folder containing multiple r5.2-* subfolders.
    :return: DataFrame with an 'insider' column.
    """

    # Locate the user's insider file
    insider_file = find_insider_answers_file(user, insider_root)

    # If no insider file exists for the user, mark all weeks as 0 (not insider)
    if not insider_file:
        df["insider"] = 0
        return df

    # Extract weeks from the insider CSV file
    insider_weeks = extract_weeks_from_csv(insider_file)

    # Label insider weeks in the user's dataframe
    df["insider"] = df["week"].apply(lambda w: 1 if w in insider_weeks else 0)

    return df


def combine_user_feature_data(user, dataset_path, insider_root):
    # Get data from different feature functions
    logon_data = get_user_logon_data(user, dataset_path)
    user_pc = get_user_pc(logon_data)
    num_other_pc = get_num_other_PC_per_week(user, user_pc, logon_data)
    after_hours_logons = get_after_hours_logons(logon_data, user)

    exe_data = get_user_exe_data(user, dataset_path)
    num_exe_files = get_num_exe_per_week(user, exe_data)

    usb_data = get_user_usb_data(user, dataset_path)
    num_usb = get_num_usb_insertions_per_week(user, usb_data)

    # Extract relevant columns
    after_hours_df = after_hours_logons[["week", "after_hours_logons"]]
    exe_df         = num_exe_files[["week", "num_exe_files"]]
    usb_df         = num_usb[["week", "num_usb_insertions"]]
    other_pc_df    = num_other_pc[["week", "num_other_pc"]]

    # Merge all dataframes on "week" using an outer join
    merged_df = after_hours_df.merge(exe_df, on="week", how="outer") \
                              .merge(usb_df, on="week", how="outer") \
                              .merge(other_pc_df, on="week", how="outer")

    # Replace NaN with 0 in all feature columns
    merged_df.fillna(0, inplace=True)

    # Add user column
    merged_df.insert(0, "user", user)
    labeled_df = label_insider_weeks(merged_df, user, insider_root)
    return labeled_df

# Example usage
dataset_path = os.path.join("Insider threat dataset", "r5.2")
user = "NIV1608"
file_path = "/content/NIV1608.csv"  # Change to your actual file path
final_df = pd.read_csv(file_path)
# final_df = combine_user_feature_data(user, data
print(final_df)

       user     week  after_hours_logons  num_exe_files  num_usb_insertions  \
0   NIV1608  2010-01                   3              0                 0.0   
1   NIV1608  2010-02                   2              0                 0.0   
2   NIV1608  2010-03                   4              0                 0.0   
3   NIV1608  2010-04                   1              0                 0.0   
4   NIV1608  2010-05                   2              0                 0.0   
..      ...      ...                 ...            ...                 ...   
56  NIV1608  2011-05                   5              0                 7.0   
57  NIV1608  2011-06                   7              0                 7.0   
58  NIV1608  2011-07                   3              0                 4.0   
59  NIV1608  2011-08                   1              0                 0.0   
60  NIV1608  2011-09                   0              0                 0.0   

    num_other_pc  insider  
0              0       

In [98]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

#  Select Features for Training
feature_cols = ["after_hours_logons", "num_exe_files", "num_usb_insertions", "num_other_pc"]
X = final_df[feature_cols]
y_true = final_df["insider"]  # Ground truth (0 = normal, 1 = anomaly)

# Normalize the Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#  Train One-Class SVM
oc_svm = OneClassSVM(kernel="rbf", gamma="scale", nu=0.9)  # Adjust nu for anomaly proportion
oc_svm.fit(X_scaled)

# Predict Anomalies
decision_scores = oc_svm.decision_function(X_scaled)  # Get decision function scores
threshold = np.percentile(decision_scores, 90)  # Detect top 10% as anomalies
y_pred = [1 if score < threshold else 0 for score in decision_scores]

#  Calculate Evaluation Metrics
precision = precision_score(y_true, y_pred, pos_label=1)
recall = recall_score(y_true, y_pred, pos_label=1)
f1 = f1_score(y_true, y_pred, pos_label=1)

#  Print Results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Precision: 0.0488
Recall: 1.0000
F1-score: 0.0930


In [99]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor  # Import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score

# Define your possible contamination values (thresholds)
contamination_values = [0.01, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15]

# Loop through each contamination value and calculate performance metrics
for contamination in contamination_values:
    lof = LocalOutlierFactor(n_neighbors=20, contamination=contamination)

    # Fit LOF model and get predictions
    y_pred = lof.fit_predict(X)  # Assuming X_train is your feature matrix
    y_pred = [1 if pred == -1 else 0 for pred in y_pred]  # Convert LOF output (-1 for outliers, 1 for inliers)

    # Calculate precision, recall, and F1 score for insider=1 (malicious)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Contamination = {contamination}: Precision = {precision:.4f}, Recall = {recall:.4f}, F1 Score = {f1:.4f}")

Contamination = 0.01: Precision = 1.0000, Recall = 0.5000, F1 Score = 0.6667
Contamination = 0.025: Precision = 1.0000, Recall = 1.0000, F1 Score = 1.0000
Contamination = 0.05: Precision = 0.6667, Recall = 1.0000, F1 Score = 0.8000
Contamination = 0.075: Precision = 0.6667, Recall = 1.0000, F1 Score = 0.8000
Contamination = 0.1: Precision = 0.6667, Recall = 1.0000, F1 Score = 0.8000
Contamination = 0.125: Precision = 0.6667, Recall = 1.0000, F1 Score = 0.8000
Contamination = 0.15: Precision = 0.2222, Recall = 1.0000, F1 Score = 0.3636
